In [136]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import math
from sklearn.metrics import f1_score

In [137]:
feature = pd.read_csv('clean_data.csv')
score = pd.read_csv('score.csv')
test = pd.read_csv('test_clean_data.csv')

In [138]:
# print(score)
feature.head()

,age,days_since_last_login,avg_time_spent,avg_transaction_value,points_in_wallet,gender_code,region_category_code,membership_category_code,year,joined_through_referral_code,preferred_offer_types_code,medium_of_operation_code,internet_option_code,used_special_discount_code,offer_application_preference_code,past_complaint_code,complaint_status_code,feedback_code
0,18,17,300.63,53005.25,781.75,0,2,3,4,1,1,0,2,1,1,0,1,4
1,32,16,306.34,12838.38,641.22,0,0,4,4,0,1,2,1,1,0,1,2,5
2,44,14,516.16,21027.00,500.69,0,1,2,5,2,1,2,2,0,1,1,3,3
3,37,11,53.27,25239.56,567.66,1,0,2,5,2,1,2,1,0,1,1,4,3
4,31,20,113.13,24483.66,663.06,0,0,2,4,1,0,3,1,0,1,1,2,3


In [139]:
feature.isnull().any()

age                                  False
days_since_last_login                False
avg_time_spent                       False
avg_transaction_value                False
points_in_wallet                     False
gender_code                          False
region_category_code                 False
membership_category_code             False
year                                 False
joined_through_referral_code         False
preferred_offer_types_code           False
medium_of_operation_code             False
internet_option_code                 False
used_special_discount_code           False
offer_application_preference_code    False
past_complaint_code                  False
complaint_status_code                False
feedback_code                        False
dtype: bool

In [140]:
feature_scaler = MinMaxScaler()
score_scaler = MinMaxScaler(feature_range=(0,5))
feature_scaler.fit(feature)
# score_scaler.fit(score)

MinMaxScaler()

In [141]:
scaled_feature = feature_scaler.transform(feature)
# scaled_score = score_scaler.transform(score)
scaled_test = feature_scaler.transform(test)

In [142]:
score

,churn_risk_score
0,2
1,1
2,5
3,5
4,5
...,...
31320,2
31321,3
31322,5
31323,3


In [143]:
scaled_feature,scaled_feature.shape

(array([[0.14814815, 0.32653061, 0.51485949, ..., 0.        , 0.25      ,
         0.5       ],
        [0.40740741, 0.30612245, 0.51580334, ..., 1.        , 0.5       ,
         0.625     ],
        [0.62962963, 0.26530612, 0.55048613, ..., 1.        , 0.75      ,
         0.375     ],
        ...,
        [0.35185185, 0.24489796, 0.35968563, ..., 0.        , 0.25      ,
         0.125     ],
        [0.7962963 , 0.28571429, 0.54494039, ..., 0.        , 0.25      ,
         0.        ],
        [0.46296296, 0.28571429, 0.4782543 , ..., 0.        , 0.25      ,
         0.625     ]]),
 (31325, 18))

In [144]:
# scaled_feature = scaled_feature.reshape(len(scaled_feature),1,18)
# scaled_score = scaled_score.reshape(len(scaled_score),1)
# scaled_test = scaled_test.reshape(len(scaled_test),1,18)

In [145]:
print(scaled_feature.shape)
# print(scaled_score.shape)
print(scaled_test.shape)

(31325, 18)
(16827, 18)


In [146]:
y = score.values
X_train, X_val, y_train, y_val = train_test_split(scaled_feature, y, test_size=0.2, random_state=42,stratify=y)

In [147]:
X_train,y_train

(array([[0.87037037, 0.12244898, 0.51934568, ..., 0.        , 0.25      ,
         0.25      ],
        [0.57407407, 0.12244898, 0.53234304, ..., 1.        , 0.        ,
         0.875     ],
        [0.74074074, 0.32653061, 0.47086714, ..., 0.        , 0.25      ,
         0.        ],
        ...,
        [0.46296296, 0.18367347, 0.50937491, ..., 0.        , 0.25      ,
         0.        ],
        [0.44444444, 0.34693878, 0.52366325, ..., 0.        , 0.25      ,
         0.125     ],
        [0.22222222, 0.18367347, 0.47715672, ..., 1.        , 0.        ,
         0.125     ]]),
 array([[5],
        [4],
        [4],
        ...,
        [4],
        [4],
        [4]], dtype=int64))

In [148]:
## endoding the y data

train_y = tf.keras.utils.to_categorical(y_train)
val_y = tf.keras.utils.to_categorical(y_val)
val_y

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [149]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(25060, 18)
(25060, 1)
(6265, 18)
(6265, 1)


In [150]:
# X_train[0].reshape(1,1,18)
## defining the generator
class generator(tf.keras.utils.Sequence):
    
    def __init__(self,batch_size,train_data,y_train):
        self.train = train_data
        self.y_train = y_train
        self.batch_size = batch_size
        self.indexes = np.arange(len(train_data))

    def __getitem__(self,i):
        start  = i*self.batch_size
        end = (i+1)* self.batch_size
        train = []
        y = []
        for j in range(start,end):
            # t_x = self.train[j].reshape(1,18)
            train.append(self.train[j])
            y.append(self.y_train[j])

        return np.array(train),np.array(y)

    def __len__(self):
        return len(self.indexes) // self.batch_size


In [151]:
x_train = generator(64,X_train,train_y)
val_data = generator(64,X_val,val_y)

In [152]:
for j in x_train:
    print(j[0].shape,j[1].shape)
    break

y_val

(64, 18) (64, 6)


array([[3],
       [3],
       [3],
       ...,
       [4],
       [2],
       [4]], dtype=int64)

In [153]:

import keras
from tensorflow.keras.models import Model, load_model

In [154]:
inputs = keras.layers.Input((18,1))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(256,activation="relu")(x)
x = keras.layers.Dense(128,activation="relu")(x)
x = keras.layers.Dense(64,activation="relu")(x)
x = keras.layers.Dense(32,activation="relu")(x)
output = keras.layers.Dense(6,activation="softmax")(x)
model = Model(inputs=inputs,outputs=output)

model.summary()




Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 48,294
Trainable params: 48,294
Non-trainable params: 0
_________________________________________________________________


In [155]:
model.compile(loss="mse",optimizer="adam",metrics=["accuracy"])

In [156]:
# model.summary()

In [157]:
epochs = 30
batch_size = 64
step_per_epochs = X_train[0]//batch_size
history = model.fit_generator(x_train,epochs=epochs,validation_data=val_data)

Epoch 1/30


C:\Users\kyush\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


391/391 [==============================] - 1s 3ms/step - loss: 0.1089 - accuracy: 0.4571 - val_loss: 0.1000 - val_accuracy: 0.5190
Epoch 2/30
391/391 [==============================] - 1s 3ms/step - loss: 0.0933 - accuracy: 0.5646 - val_loss: 0.0881 - val_accuracy: 0.58390s - loss: 0
Epoch 3/30
391/391 [==============================] - 1s 3ms/step - loss: 0.0843 - accuracy: 0.6012 - val_loss: 0.0835 - val_accuracy: 0.6047
Epoch 4/30
391/391 [==============================] - 1s 3ms/step - loss: 0.0816 - accuracy: 0.6105 - val_loss: 0.0809 - val_accuracy: 0.5952
Epoch 5/30
391/391 [==============================] - 1s 2ms/step - loss: 0.0799 - accuracy: 0.6154 - val_loss: 0.0817 - val_accuracy: 0.6136
Epoch 6/30
391/391 [==============================] - 1s 3ms/step - loss: 0.0787 - accuracy: 0.6215 - val_loss: 0.0778 - val_accuracy: 0.6258
Epoch 7/30
391/391 [==============================] - 1s 3ms/step - loss: 0.0771 - accuracy: 0.6313 - val_loss: 0.0772 - val_accuracy: 0.6339
Epoch

In [158]:
# sns.lineplot(data=history.history,x=np.arange(epochs),y='mse')
# sns.lineplot(data=history.history,x=np.arange(epochs),y='val_mse')
# sns.lineplot(data=history.history)

In [159]:
# model.evaluate(X_val,y_val)

In [160]:
# model.predict(X_test).shape

In [161]:
# y_val.shape

In [162]:
# sns.lineplot(data=d)

In [163]:
# model.predict(scaled_test[:10])



In [164]:
# model.save('trained_model')

In [165]:
input_data=[ 24,16,32.42,37953.61,614.71,1,0,1,5,2,2,2,0,1,0,1,4,2]

In [166]:
 input_data_as_numpy_array = np.asarray(input_data)


In [167]:
 input_data_reshaped =  input_data_as_numpy_array.reshape(1,-1)

In [168]:
 prediction = model.predict(scaled_test[:1])

In [169]:
 print(prediction)

[[2.2153567e-07 9.6203329e-04 4.6624511e-04 9.7717154e-01 1.8944187e-02
  2.4558357e-03]]


In [180]:
if(prediction.all()  >2.1 ):
    print('The customer will not churn')
else:
    print('The customer will  churn')

The customer will  churn
